## Imports

In [1]:
import numpy as np
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
import os
import cv2
from  matplotlib import pyplot as plt
import tensorflow as tf

## Constants

In [2]:
img_folder="dataset"
IMG_HEIGHT=300
IMG_WIDTH=300

## Define the standalone discriminator model

In [3]:
def define_discriminator(in_shape=(300,300,3)):
    model = Sequential()
    # downsample
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # classifier
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

## Define the standalone generator model

In [4]:
def define_generator(latent_dim):
    model = Sequential()
    # foundation for 75*75 image
    n_nodes = 128 * 75 * 75
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((75, 75, 128)))
    # upsample to 14*14
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 28*28
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # generate
    model.add(Conv2D(3, (7,7), activation='tanh', padding='same'))
    return model

## define the combined generator and discriminator model, for updating the generator

In [5]:
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    # connect them 
    model = Sequential()
    # add the generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

## load images

In [6]:
def create_dataset(img_folder):
    img_data_array=[]
    class_name=[]
   
    for dir1 in os.listdir(img_folder):
        i = 0
        for file in os.listdir(os.path.join(img_folder, dir1)):
            image_path= os.path.join(img_folder, dir1,  file)
            image= cv2.imread(image_path, cv2.COLOR_BGR2RGB)
            image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
            image=np.array(image)
            image = image.astype('float32')
            image = (image - 127.5) / 127.5
            img_data_array.append(image)
            class_name.append(dir1)
            i += 1
#             print(i)
            if i == 10:
                break
#     , np.array(class_name)
    return np.array(img_data_array)

## select real samples

In [7]:
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # select images
    X = dataset[ix]
    # generate class labels
    y = ones((n_samples, 1))
    return X, y

## generate points in latent space as input for the generator

In [8]:
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

## use the generator to generate n fake examples, with class labels

In [9]:
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = generator.predict(x_input)
    # create class labels
    y = zeros((n_samples, 1))
    return X, y

## train the generator and discriminator

In [10]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=2, n_batch=4):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # update discriminator model weights
#             print(X_real)
#             print(y_real)
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            # generaten 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update discriminator model weights
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # summarize loss on this batch
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' % 
                  (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
    # save the generator model
    g_model.save('generator.h5')

In [11]:
# size of the latent space 
latent_dim = 100
# create the discriminator
discriminator = define_discriminator()
# print(discriminator.summary())
# create the generator
generator = define_generator(latent_dim)
# print(generator.summary())
# create the gan
gan_model = define_gan(generator, discriminator)
# load image data
dataset = create_dataset(img_folder)
# train model
with tf.device("cpu:0"):
    train(generator, discriminator, gan_model, dataset, latent_dim)

<class 'numpy.ndarray'>
>1, 1/5, d1=0.707, d2=0.697 g=0.698
>1, 2/5, d1=0.061, d2=0.703 g=0.702
>1, 3/5, d1=0.006, d2=0.703 g=0.711
>1, 4/5, d1=0.001, d2=0.699 g=0.747
>1, 5/5, d1=0.004, d2=0.646 g=0.873
>2, 1/5, d1=0.001, d2=0.491 g=1.168
>2, 2/5, d1=2.638, d2=0.548 g=0.791
>2, 3/5, d1=0.000, d2=0.699 g=0.783
>2, 4/5, d1=0.000, d2=0.751 g=0.774
>2, 5/5, d1=0.000, d2=0.950 g=0.807
